# Double Heston + Jumps Calibration - Neural Network Training
## Google Colab Training Notebook

**Setup Instructions:**
1. Runtime → Change runtime type → **T4 GPU**
2. Run Cell 1 to upload `synthetic_10k.pkl`
3. Run all cells sequentially
4. Download trained model files at the end

**Expected Training Time:** ~30 minutes on T4 GPU

---
## Cell 1: Upload Dataset

In [ ]:
from google.colab import files
import os

print("📤 Please upload synthetic_10k.pkl (2.1 MB)")
print("   This file contains 10,000 synthetic option price samples\n")

uploaded = files.upload()

if 'synthetic_10k.pkl' not in uploaded:
    raise FileNotFoundError("❌ synthetic_10k.pkl not found. Please upload it.")

print("\n✓ File uploaded successfully!")
print(f"  Size: {os.path.getsize('synthetic_10k.pkl') / 1024 / 1024:.2f} MB")

---
## Cell 2: Install Dependencies

In [ ]:
!pip install numpy scipy tensorflow scikit-learn matplotlib -q

import numpy as np
import tensorflow as tf
print(f"✓ TensorFlow {tf.__version__}")
print(f"✓ NumPy {np.__version__}")
print(f"✓ GPU Available: {tf.config.list_physical_devices('GPU')}")

---
## Cell 3: Define FFN Architecture and Preprocessing

In [ ]:
import pickle
from tensorflow import keras
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

class OptionDataPreprocessor:
    """
    Prepare option price data for neural network training
    """
    
    def __init__(self):
        self.feature_scaler = None
        self.target_scaler = None
        self.param_names = [
            'v1_0', 'kappa1', 'theta1', 'sigma1', 'rho1',
            'v2_0', 'kappa2', 'theta2', 'sigma2', 'rho2',
            'lambda_j', 'mu_j', 'sigma_j'
        ]
    
    def extract_features(self, option_prices, strikes, maturities, spot=100.0):
        """
        Extract 10 meaningful features from 15 raw option prices:
        - 3 features per maturity (ATM, skew, butterfly)
        - Term structure slope
        - Total ATM premium
        """
        n_samples = option_prices.shape[0]
        n_strikes = len(strikes)
        n_maturities = len(maturities)
        
        # Reshape: (n_samples, n_strikes, n_maturities)
        prices_3d = option_prices.reshape(n_samples, n_strikes, n_maturities)
        
        features_list = []
        
        for i in range(n_samples):
            sample_features = []
            
            for mat_idx, T in enumerate(maturities):
                prices_at_maturity = prices_3d[i, :, mat_idx]
                
                # Feature 1: ATM price (normalized by spot)
                atm_idx = np.argmin(np.abs(strikes - spot))
                atm_price = prices_at_maturity[atm_idx]
                sample_features.append(atm_price / spot)
                
                # Feature 2: Skew (25-delta risk reversal approximation)
                otm_call_idx = np.argmin(np.abs(strikes - spot*1.05))
                otm_put_idx = np.argmin(np.abs(strikes - spot*0.95))
                skew = (prices_at_maturity[otm_call_idx] - 
                       prices_at_maturity[otm_put_idx]) / spot
                sample_features.append(skew)
                
                # Feature 3: Curvature (butterfly)
                itm_idx = np.argmin(np.abs(strikes - spot*0.95))
                otm_idx = np.argmin(np.abs(strikes - spot*1.05))
                butterfly = (prices_at_maturity[itm_idx] + 
                           prices_at_maturity[otm_idx] - 
                           2 * atm_price) / spot
                sample_features.append(butterfly)
            
            # Feature 4: Term structure slope
            if n_maturities > 1:
                atm_short = prices_3d[i, atm_idx, 0]
                atm_long = prices_3d[i, atm_idx, -1]
                term_slope = (atm_long - atm_short) / spot
                sample_features.append(term_slope)
            
            # Feature 5: Total ATM premium across maturities
            total_atm = np.sum(prices_3d[i, atm_idx, :]) / spot
            sample_features.append(total_atm)
            
            features_list.append(sample_features)
        
        return np.array(features_list)
    
    def prepare_training_data(self, data_path, test_size=0.15, val_size=0.15):
        """
        Load synthetic data and prepare train/val/test splits
        """
        # Load data
        print("Loading synthetic data...")
        with open(data_path, 'rb') as f:
            data = pickle.load(f)
        
        # Extract features
        print("Extracting features from option prices...")
        features = self.extract_features(
            data['option_prices'],
            data['strikes'],
            data['maturities']
        )
        
        targets = data['parameters']
        
        print(f"Feature shape: {features.shape}")
        print(f"Target shape: {targets.shape}")
        
        # Split data
        X_train, X_temp, y_train, y_temp = train_test_split(
            features, targets, test_size=(test_size + val_size), random_state=42
        )
        
        val_ratio = val_size / (test_size + val_size)
        X_val, X_test, y_val, y_test = train_test_split(
            X_temp, y_temp, test_size=(1 - val_ratio), random_state=42
        )
        
        # Normalize features
        print("Normalizing features...")
        self.feature_scaler = StandardScaler()
        X_train_scaled = self.feature_scaler.fit_transform(X_train)
        X_val_scaled = self.feature_scaler.transform(X_val)
        X_test_scaled = self.feature_scaler.transform(X_test)
        
        # Transform targets (log for positive parameters)
        print("Normalizing targets...")
        y_train_transformed = self._transform_targets(y_train)
        y_val_transformed = self._transform_targets(y_val)
        y_test_transformed = self._transform_targets(y_test)
        
        self.target_scaler = StandardScaler()
        y_train_scaled = self.target_scaler.fit_transform(y_train_transformed)
        y_val_scaled = self.target_scaler.transform(y_val_transformed)
        y_test_scaled = self.target_scaler.transform(y_test_transformed)
        
        print(f"\nData splits:")
        print(f"  Train: {X_train_scaled.shape[0]} samples")
        print(f"  Val:   {X_val_scaled.shape[0]} samples")
        print(f"  Test:  {X_test_scaled.shape[0]} samples")
        
        return {
            'train': (X_train_scaled, y_train_scaled),
            'val': (X_val_scaled, y_val_scaled),
            'test': (X_test_scaled, y_test_scaled),
            'y_train_orig': y_train,
            'y_val_orig': y_val,
            'y_test_orig': y_test
        }
    
    def _transform_targets(self, targets):
        """Apply log transform to positive parameters"""
        transformed = targets.copy()
        log_indices = [0, 1, 2, 3, 5, 6, 7, 8, 10, 12]
        transformed[:, log_indices] = np.log(transformed[:, log_indices] + 1e-10)
        return transformed
    
    def inverse_transform_predictions(self, y_scaled):
        """Convert scaled predictions back to original parameter space"""
        y_transformed = self.target_scaler.inverse_transform(y_scaled)
        y_original = y_transformed.copy()
        log_indices = [0, 1, 2, 3, 5, 6, 7, 8, 10, 12]
        y_original[:, log_indices] = np.exp(y_transformed[:, log_indices])
        return y_original


def ffn_model(input_dim, output_dim):
    """Build FFN architecture: 512→256→128→64 with BatchNorm and Dropout"""
    from tensorflow.keras import layers
    
    input_layer = layers.Input(shape=(input_dim,), name='option_features')
    
    x = layers.Dense(512, activation='relu')(input_layer)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.3)(x)
    
    x = layers.Dense(256, activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.3)(x)
    
    x = layers.Dense(128, activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.2)(x)
    
    x = layers.Dense(64, activation='relu')(x)
    
    output_layer = layers.Dense(output_dim, activation='linear', name='model_output')(x)
    
    model = keras.Model(inputs=input_layer, outputs=output_layer)
    return model

print("✓ FFN architecture and preprocessing defined")

---
## Cell 4: Load and Prepare Data

In [ ]:
print("="*70)
print("PREPARING DATA FOR TRAINING")
print("="*70 + "\n")

preprocessor = OptionDataPreprocessor()
data_splits = preprocessor.prepare_training_data('synthetic_10k.pkl')

X_train, y_train = data_splits['train']
X_val, y_val = data_splits['val']
X_test, y_test = data_splits['test']

print("\n✓ Data preparation complete!")
print(f"  Input features:  {X_train.shape[1]} dimensions")
print(f"  Output targets:  {y_train.shape[1]} parameters")

---
## Cell 5: Build and Compile Model

In [ ]:
print("="*70)
print("BUILDING NEURAL NETWORK MODEL")
print("="*70 + "\n")

input_dim = X_train.shape[1]
output_dim = y_train.shape[1]

model = ffn_model(input_dim=input_dim, output_dim=output_dim)

model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    loss='mse',
    metrics=['mae', 'mape']
)

print(f"Architecture: {input_dim} → [512, 256, 128, 64] → {output_dim}")
print(f"Total parameters: {model.count_params():,}")
print("\n" + "="*70)
model.summary()
print("="*70)

---
## Cell 6: Train Model (~ 30 minutes on T4 GPU)

In [ ]:
print("\n" + "="*70)
print("STARTING TRAINING")
print("="*70 + "\n")

# Callbacks
callbacks_list = [
    keras.callbacks.EarlyStopping(
        monitor='val_loss',
        patience=15,
        restore_best_weights=True,
        verbose=1
    ),
    keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=7,
        min_lr=1e-6,
        verbose=1
    ),
    keras.callbacks.ModelCheckpoint(
        'best_model.keras',
        monitor='val_loss',
        save_best_only=True,
        verbose=1
    )
]

# Train
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=100,
    batch_size=256,
    callbacks=callbacks_list,
    verbose=1
)

print("\n" + "="*70)
print("✓ TRAINING COMPLETE!")
print("="*70)

---
## Cell 7: Evaluate on Test Set

In [ ]:
print("\n" + "="*70)
print("EVALUATING ON TEST SET")
print("="*70 + "\n")

# Predict on test set
y_pred_scaled = model.predict(X_test, verbose=0)
y_pred = preprocessor.inverse_transform_predictions(y_pred_scaled)
y_true = data_splits['y_test_orig']

# Compute per-parameter errors
param_names = preprocessor.param_names

print(f"{'Parameter':<12} {'MAE':<12} {'RMSE':<12} {'MAPE %':<10}")
print("-" * 50)

for i, name in enumerate(param_names):
    mae = np.mean(np.abs(y_true[:, i] - y_pred[:, i]))
    rmse = np.sqrt(np.mean((y_true[:, i] - y_pred[:, i])**2))
    mape = np.mean(np.abs((y_true[:, i] - y_pred[:, i]) / (y_true[:, i] + 1e-10))) * 100
    
    print(f"{name:<12} {mae:<12.6f} {rmse:<12.6f} {mape:<10.2f}")

# Overall metrics
mae_overall = np.mean(np.abs(y_true - y_pred))
rmse_overall = np.sqrt(np.mean((y_true - y_pred)**2))
mape_overall = np.mean(np.abs((y_true - y_pred) / (y_true + 1e-10))) * 100

print("-" * 50)
print(f"{'OVERALL':<12} {mae_overall:<12.6f} {rmse_overall:<12.6f} {mape_overall:<10.2f}")
print("\n" + "="*70)

---
## Cell 8: Plot Training History

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Loss curve
axes[0].plot(history.history['loss'], label='Train Loss', linewidth=2)
axes[0].plot(history.history['val_loss'], label='Val Loss', linewidth=2)
axes[0].set_xlabel('Epoch', fontsize=12)
axes[0].set_ylabel('Loss (MSE)', fontsize=12)
axes[0].set_title('Training and Validation Loss', fontsize=14, fontweight='bold')
axes[0].legend(fontsize=10)
axes[0].grid(True, alpha=0.3)

# MAE curve
axes[1].plot(history.history['mae'], label='Train MAE', linewidth=2)
axes[1].plot(history.history['val_mae'], label='Val MAE', linewidth=2)
axes[1].set_xlabel('Epoch', fontsize=12)
axes[1].set_ylabel('MAE', fontsize=12)
axes[1].set_title('Training and Validation MAE', fontsize=14, fontweight='bold')
axes[1].legend(fontsize=10)
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('training_history.png', dpi=300, bbox_inches='tight')
plt.show()

print("✓ Training history plot saved to 'training_history.png'")

---
## Cell 9: Save Scalers and History

In [ ]:
# Save scalers
with open('scalers.pkl', 'wb') as f:
    pickle.dump({
        'feature_scaler': preprocessor.feature_scaler,
        'target_scaler': preprocessor.target_scaler,
        'log_indices': [0, 1, 2, 3, 5, 6, 7, 8, 10, 12]
    }, f)

print("✓ Scalers saved to 'scalers.pkl'")

# Save training history
with open('training_history.pkl', 'wb') as f:
    pickle.dump(history.history, f)

print("✓ Training history saved to 'training_history.pkl'")

# Check files
import os
print("\nFiles ready for download:")
for fname in ['best_model.keras', 'scalers.pkl', 'training_history.pkl', 'training_history.png']:
    if os.path.exists(fname):
        size_kb = os.path.getsize(fname) / 1024
        print(f"  ✓ {fname:<30} ({size_kb:.1f} KB)")
    else:
        print(f"  ✗ {fname:<30} (NOT FOUND)")

---
## Cell 10: Download All Files

In [ ]:
from google.colab import files
import os
import time

print("📥 DOWNLOADING FILES ONE BY ONE")
print("="*70 + "\n")

# Download files individually with delays
files_to_download = ['best_model.keras', 'scalers.pkl', 'training_history.pkl', 'training_history.png']

for i, fname in enumerate(files_to_download, 1):
    if os.path.exists(fname):
        size_kb = os.path.getsize(fname) / 1024
        print(f"[{i}/4] Downloading {fname} ({size_kb:.1f} KB)...")
        files.download(fname)
        time.sleep(2)  # Wait between downloads
        print(f"     ✓ {fname} downloaded!\n")
    else:
        print(f"[{i}/4] ✗ {fname} NOT FOUND - skipping\n")

print("="*70)
print("✓ DOWNLOAD COMPLETE!")
print("="*70)
print("\n📋 Next steps:")
print("  1. Check your Downloads folder for these files:")
print("     - best_model.keras")
print("     - scalers.pkl")
print("     - training_history.pkl")
print("     - training_history.png")
print("\n  2. Rename: best_model.keras → best_ffn_model.keras")
print("\n  3. Move all files to: /Users/zen/double-heston-calibrator/")
print("\n  4. Run fine-tuning:")
print("     python3 finetune_ffn_on_lbfgs.py")
print("="*70)

---
## Cell 11: Alternative Download (ZIP file) - Use if Cell 10 fails

In [ ]:
import zipfile
import os
from google.colab import files

print("📦 Creating ZIP archive with all files...")
print("="*70 + "\n")

# Create zip file
zip_filename = 'trained_model_files.zip'
with zipfile.ZipFile(zip_filename, 'w', zipfile.ZIP_DEFLATED) as zipf:
    files_to_zip = ['best_model.keras', 'scalers.pkl', 'training_history.pkl', 'training_history.png']
    
    for fname in files_to_zip:
        if os.path.exists(fname):
            size_kb = os.path.getsize(fname) / 1024
            zipf.write(fname)
            print(f"  ✓ Added {fname} ({size_kb:.1f} KB)")
        else:
            print(f"  ✗ {fname} NOT FOUND - skipping")

zip_size_mb = os.path.getsize(zip_filename) / 1024 / 1024
print(f"\n✓ ZIP created: {zip_filename} ({zip_size_mb:.2f} MB)")

print("\n📥 Downloading ZIP file...")
print("="*70 + "\n")

files.download(zip_filename)

print("\n" + "="*70)
print("✓ DOWNLOAD COMPLETE!")
print("="*70)
print("\n📋 Next steps:")
print("  1. Extract trained_model_files.zip")
print("  2. You'll get these files:")
print("     - best_model.keras")
print("     - scalers.pkl")
print("     - training_history.pkl")
print("     - training_history.png")
print("\n  3. Rename: best_model.keras → best_ffn_model.keras")
print("\n  4. Move all files to: /Users/zen/double-heston-calibrator/")
print("\n  5. Run fine-tuning:")
print("     python3 finetune_ffn_on_lbfgs.py")
print("="*70)